In [1]:
!bash /home/azureuser/cloudfiles/code/blobfuse/blobfuse_raadsinformatie.sh

In [2]:
import sys
sys.path.append("..")

# Select where to run notebook: "azure" or "local"
my_run = "azure"

# import my_secrets as sc
# import settings as st

if my_run == "azure":
    import config_azure as cf
elif my_run == "local":
    import config as cf


import os
if my_run == "azure":
    if not os.path.exists(cf.HUGGING_CACHE):
        os.mkdir(cf.HUGGING_CACHE)
    os.environ["TRANSFORMERS_CACHE"] = cf.HUGGING_CACHE

# set-up environment - GEITje-7b-chat InContextLearning:
# - install blobfuse -> sudo apt-get install blobfuse
# - pip install transformers
# - pip install torch
# - pip install accelerate
# - pip install jupyter
# - pip install ipywidgets

import pandas as pd

## Notebook Overview
Goal: repair responses for fine-tuned Mistral.

#### Check for ALL prediction files if predictions were correctly extracted
The Regex pattern that's used to extract the prediction from the response has been updated, after running the experiments. I found out that the regex pattern, which just matches everything within {} was not strict enoug, thus the pattern was adjusted to only match when 'categorie' is name within the curly brackets. This means all predictions need to be checked on errors when extracting with previous regex pattern.

In [6]:
sys.path.append('../scripts/') 
import prediction_helperfunctions as pt

def extract_predictions_new_regex(list_of_dfs):
    list_of_new_dfs = []
    errors = 0
    for df in list_of_dfs:
        df['new_prediction'] = df['response'].apply(pt.get_prediction_from_response)
        if len(df.loc[df['new_prediction']!=df['prediction']]) > 0:
            print(f"Following run_id has new predictions: {set(df['run_id'])}")
            errors += 1

    if errors == 0:
        print('All dataframes are good!')

    else:
        print(f"There are {errors} dataframes with problems.")

##### GEITje

In [7]:
# IC
g_fewshot = pd.read_pickle(f"{cf.output_path}/predictionsFinal/in_context/GEITje/fewshot_prompt_no_template/First200Last0Predictions.pkl")
g_ic_100 = pd.read_pickle(f"{cf.output_path}/predictionsFinal/in_context/GEITje/zeroshot_prompt_geitje/First100Last0Predictions.pkl")
g_ic_200 = pd.read_pickle(f"{cf.output_path}/predictionsFinal/in_context/GEITje/zeroshot_prompt_geitje/First200Last0Predictions.pkl")
g_ic_100_100 = pd.read_pickle(f"{cf.output_path}/predictionsFinal/in_context/GEITje/zeroshot_prompt_geitje/First100Last100Predictions.pkl")

extract_predictions_new_regex([g_fewshot, g_ic_100, g_ic_200, g_ic_100_100])

All dataframes are good!


In [28]:
pred1 = pd.read_pickle(f"{cf.output_path}/predictionsFinal/finetuning/1epochs/MistralFirst200Last0Predictions.pkl")
print(f" Epoch 1 has {len(pred1[~pred1['response'].str.startswith('{')])} responses that do not start with {'{'}")
print(f" Epoch 1 has {len(pred1.loc[pred1['prediction']=='NoPredictionFormat'])} predictions that are NoPredictionFormat errors.")
pred2 = pd.read_pickle(f"{cf.output_path}/predictionsFinal/finetuning/2epochs/MistralFirst200Last0Predictions.pkl")
print(f" Epoch 2 has {len(pred2[~pred2['response'].str.startswith('{')])} responses that do not start with {'{'}")
print(f" Epoch 2 has {len(pred2.loc[pred2['prediction']=='NoPredictionFormat'])} predictions that are NoPredictionFormat errors.")
pred3 = pd.read_pickle(f"{cf.output_path}/predictionsFinal/finetuning/3epochs/MistralFirst200Last0Predictions.pkl")
print(f" Epoch 3 has {len(pred3[~pred3['response'].str.startswith('{')])} responses that do not start with {'{'}")
print(f" Epoch 3 has {len(pred3.loc[pred3['prediction']=='NoPredictionFormat'])} predictions that are NoPredictionFormat errors.")


 Epoch 1 has 0 responses that do not start with {
 Epoch 1 has 0 predictions that are NoPredictionFormat errors.
 Epoch 2 has 1100 responses that do not start with {
 Epoch 2 has 1015 predictions that are NoPredictionFormat errors.
 Epoch 3 has 1100 responses that do not start with {
 Epoch 3 has 1097 predictions that are NoPredictionFormat errors.


In [36]:
import re
import sys
sys.path.append('../scripts/') 
import prompt_template as pt
from collections import Counter

def get_prediction_from_response(response):
    # get a list of the possible classes
    classes_list = pt.get_class_list()
    

    # check if part of string matches given output format to prompt -> Llama is annoying and won't only give the output format :(
    pattern = r'\{[^{}]*categorie[^{}]*\}'
    matches = re.findall(pattern, response)
    if len(matches) == 1:
        prediction_output = matches[0]
        predictions = [True if category.lower() in prediction_output.lower() else False for category in classes_list]

        # check if multiple classes were named, this is a prediction error
        if Counter(predictions)[True] > 1:
            return "MultiplePredictionErrorInOutput"

        # check if exactly one class is named, this is the prediction
        elif Counter(predictions)[True] == 1:
            prediction = [category.lower() for category in classes_list if category.lower() in prediction_output.lower()]
            return prediction[0]

        # if no class is named, then this is a no prediction error
        else:
            return 'NoPredictionInOutput'
        
    elif len(matches) > 1:
        return 'MultiplePredictionErrorInFormatting'
    else:
        return 'NoPredictionFormat'
    
pred2['new_prediction'] = pred2['response'].apply(get_prediction_from_response)
display(pred2.loc[pred2['new_prediction']!= pred2['prediction']])

,id,path,text_column,prompt_function,response,prediction,label,runtime,date,prompt,run_id,train_set,test_set,shots,complete_regex,new_prediction
4,16341,/home/azureuser/cloudfiles/code/blobfuse/raads...,TruncationLlamaTokensFront200Back0,zeroshot_prompt_mistral_llama,"""agenda"": Actualiteit, Raadsadres, Schriftelij...",actualiteit,agenda,360.445917,2024-06-04 19:56:07.571265+02:00,Classificeer het document in één van de catego...,FT_AmsterdamDocClassificationMistral200T2Epoch...,train,test,0,0,NoPredictionFormat
1,15876,/home/azureuser/cloudfiles/code/blobfuse/raads...,TruncationLlamaTokensFront200Back0,zeroshot_prompt_mistral_llama,"""agenda"": Actualiteit, Factsheet, Motie, Schri...",MultiplePredictionErrorInFormatting,agenda,361.297196,2024-06-04 20:27:29.568910+02:00,Classificeer het document in één van de catego...,FT_AmsterdamDocClassificationMistral200T2Epoch...,train,test,0,0,NoPredictionFormat
3,32993,/home/azureuser/cloudfiles/code/blobfuse/raads...,TruncationLlamaTokensFront200Back0,zeroshot_prompt_mistral_llama,"""factsheet"": Factsheet ouderenhuisvesting 2019...",NoPredictionInOutput,factsheet,233.029459,2024-06-04 20:40:00.667747+02:00,Classificeer het document in één van de catego...,FT_AmsterdamDocClassificationMistral200T2Epoch...,train,test,0,0,NoPredictionFormat
3,13141,/home/azureuser/cloudfiles/code/blobfuse/raads...,TruncationLlamaTokensFront200Back0,zeroshot_prompt_mistral_llama,"""agenda"": Agenda}\n{\n""Aanvang"": ""19.00"",\n""Lo...",NoPredictionInOutput,agenda,359.416345,2024-06-04 21:47:52.470449+02:00,Classificeer het document in één van de catego...,FT_AmsterdamDocClassificationMistral200T2Epoch...,train,test,0,0,NoPredictionFormat
7,13810,/home/azureuser/cloudfiles/code/blobfuse/raads...,TruncationLlamaTokensFront200Back0,zeroshot_prompt_mistral_llama,"""agenda"": Agenda Open Raad 8 juni 2011\n\n""typ...",MultiplePredictionErrorInFormatting,agenda,360.024872,2024-06-04 22:07:30.030797+02:00,Classificeer het document in één van de catego...,FT_AmsterdamDocClassificationMistral200T2Epoch...,train,test,0,0,NoPredictionFormat
0,26240,/home/azureuser/cloudfiles/code/blobfuse/raads...,TruncationLlamaTokensFront200Back0,zeroshot_prompt_mistral_llama,"""agenda"": Agenda\n} ""type"": Agenda\n} ""items"":...",MultiplePredictionErrorInFormatting,agenda,275.279551,2024-06-04 23:32:21.025961+02:00,Classificeer het document in één van de catego...,FT_AmsterdamDocClassificationMistral200T2Epoch...,train,test,0,0,NoPredictionFormat
1,9729,/home/azureuser/cloudfiles/code/blobfuse/raads...,TruncationLlamaTokensFront200Back0,zeroshot_prompt_mistral_llama,"""agenda"": Actualiteit, Raadsadres, Motie, Voor...",MultiplePredictionErrorInFormatting,actualiteit,362.618332,2024-06-04 23:42:06.982890+02:00,Classificeer het document in één van de catego...,FT_AmsterdamDocClassificationMistral200T2Epoch...,train,test,0,0,NoPredictionFormat
7,14645,/home/azureuser/cloudfiles/code/blobfuse/raads...,TruncationLlamaTokensFront200Back0,zeroshot_prompt_mistral_llama,"""agenda"": {\n""type"": ""Agenda"",\n""id"": ""1"",\n""n...",MultiplePredictionErrorInFormatting,agenda,360.392639,2024-06-05 01:01:47.248914+02:00,Classificeer het document in één van de catego...,FT_AmsterdamDocClassificationMistral200T2Epoch...,train,test,0,1,NoPredictionInOutput
4,13649,/home/azureuser/cloudfiles/code/blobfuse/raads...,TruncationLlamaTokensFront200Back0,zeroshot_prompt_mistral_llama,"""agenda"": {\n""type"": ""Agenda"",\n""commissie"": ""...",agenda,agenda,392.431147,2024-06-05 01:26:00.353349+02:00,Classificeer het document in één van de catego...,FT_AmsterdamDocClassificationMistral200T2Epoch...,train,test,0,0,NoPredictionFormat
5,14960,/home/azureuser/cloudfiles/code/blobfuse/raads...,TruncationLlamaTokensFront200Back0,zeroshot_prompt_mistral_llama,"""agenda"": Agenda\n\n""type"": Agenda\n\n""items"":...",MultiplePredictionErrorInFormatting,agenda,335.768803,2024-06-05 03:06:36.001185+02:00,Classificeer het document in één van de catego...,FT_

In [34]:
import re
from collections import Counter
pattern = r'\{[^{}]*categorie[^{}]*\}'
pred1['complete_regex'] = pred1['response'].apply(lambda x: len(re.findall(pattern, x)))
pred2['complete_regex'] = pred2['response'].apply(lambda x: len(re.findall(pattern, x)))
pred3['complete_regex'] = pred3['response'].apply(lambda x: len(re.findall(pattern, x)))
print(Counter(pred1['complete_regex']))
print(Counter(pred2['complete_regex']))
print(Counter(pred3['complete_regex']))

Counter({1: 1100})
Counter({0: 1035, 1: 65})
Counter({0: 1097, 2: 2, 1: 1})


In [19]:
yeet = pred2[~pred2['response'].str.startswith('{')]
yeet = yeet.loc[yeet['prediction']!='NoPredictionFormat']
for index, row in yeet.iterrows():
    print(row['prediction'])
    print(row['response'], '\n')
# display(yeet)

raadsnotulen
"raadsnotulen": {
"categorie": Raadsnotulen
} 

raadsnotulen
"raadsnotulen": {
"categorie": Raadsnotulen
} 

raadsnotulen
"raadsnotulen": {
"categorie": Raadsnotulen
}
} 

actualiteit
"vergadering": {
"categorie": Actualiteit
}
} 

raadsnotulen
"raadsnotulen": {
"categorie": Raadsnotulen
} 

raadsnotulen
"raadsnotulen": {
"categorie": Raadsnotulen
} 

actualiteit
"agenda": Actualiteit, Raadsadres, Schriftelijke Vraag, Voordracht, Motie, Factsheet, Onderzoeksrapport, Raadsnotulen, Agenda, Besluit
} "actualiteit": {
"nummer": 1,
"naam": "Actualiteit van de raadsleden de heer Paternotte (D66) en de heer
Van Drooge (CDA) inzake het bericht dat het college van burgemeester en
wethouders de raadscommissie voor Economische Zaken, Bedrijfsvoering en
Inkoop, Bedrijven en Deelnemingen, Personeel en Organisatie, Dienstverlening,
Luchthaven, Dierenwelzijn, Waterbeheer wil laten gaan vergaderen op
donderdag 10 mei 2012 om 19.00 uur in de Rooszaal 02.
} "raadsadres": {
"nummer": 2,
" 



In [6]:
filtered_df = pred2[~pred2['response'].str.startswith('{')]
display(filtered_df)


,id,path,text_column,prompt_function,response,prediction,label,runtime,date,prompt,run_id,train_set,test_set,shots
0,26304,/home/azureuser/cloudfiles/code/blobfuse/raads...,TruncationLlamaTokensFront200Back0,zeroshot_prompt_mistral_llama,"""raadsnotulen"": {\n""vergaderdatum"": ""15 septem...",NoPredictionFormat,raadsnotulen,394.712015,2024-06-03 12:11:00.415835+02:00,Classificeer het document in één van de catego...,FT_AmsterdamDocClassificationMistral200T2Epoch...,train,test,0
1,32939,/home/azureuser/cloudfiles/code/blobfuse/raads...,TruncationLlamaTokensFront200Back0,zeroshot_prompt_mistral_llama,"""categorie"": Onderzoeksrapport}",NoPredictionFormat,factsheet,26.393481,2024-06-03 12:11:26.810359+02:00,Classificeer het document in één van de catego...,FT_AmsterdamDocClassificationMistral200T2Epoch...,train,test,0
2,33085,/home/azureuser/cloudfiles/code/blobfuse/raads...,TruncationLlamaTokensFront200Back0,zeroshot_prompt_mistral_llama,"""categorie"": Onderzoeksrapport}",NoPredictionFormat,factsheet,26.199955,2024-06-03 12:11:53.011982+02:00,Classificeer het document in één van de catego...,FT_AmsterdamDocClassificationMistral200T2Epoch...,train,test,0
3,22985,/home/azureuser/cloudfiles/code/blobfuse/raads...,TruncationLlamaTokensFront200Back0,zeroshot_prompt_mistral_llama,"""categorie"": Brief}",NoPredictionFormat,brief,20.403498,2024-06-03 12:12:13.417111+02:00,Classificeer het document in één van de catego...,FT_AmsterdamDocClassificationMistral200T2Epoch...,train,test,0
4,32991,/home/azureuser/cloudfiles/code/blobfuse/raads...,TruncationLlamaTokensFront200Back0,zeroshot_prompt_mistral_llama,"""categorie"": Onderzoeksrapport}",NoPredictionFormat,factsheet,26.293198,2024-06-03 12:12:39.711931+02:00,Classificeer het document in één van de catego...,FT_AmsterdamDocClassificationMistral200T2Epoch...,train,test,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,13888,/home/azureuser/cloudfiles/code/blobfuse/raads...,TruncationLlamaTokensFront200Back0,zeroshot_prompt_mistral_llama,"""agenda"": Agenda voor de vergadering van de st...",NoPredictionFormat,agenda,389.155650,2024-06-05 10:25:33.290129+02:00,Classificeer het document in één van de catego...,FT_AmsterdamDocClassificationMistral200T2Epoch...,train,test,0
6,28826,/home/azureuser/cloudfiles/code/blobfuse/raads...,TruncationLlamaTokensFront200Back0,zeroshot_prompt_mistral_llama,"""categorie"": Voordracht}",NoPredictionFormat,voordracht,22.437386,2024-06-05 10:25:55.729447+02:00,Classificeer het document in één van de catego...,FT_AmsterdamDocClassificationMistral200T2Epoch...,train,test,0
7,25722,/home/azureuser/cloudfiles/code/blobfuse/raads...,TruncationLlamaTokensFront200Back0,zeroshot_prompt_mistral_llama,"""agenda"": Agenda\n\n""type"": Agenda\n}",NoPredictionFormat,agenda,30.000683,2024-06-05 10:26:25.731761+02:00,Classificeer het document in één van de catego...,FT_AmsterdamDocClassificationMistral200T2Epoch...,train,test,0
8,23998,/home/azureuser/cloudfiles/code/blobfuse/raads...,TruncationLlamaTokensFront200Back0,zeroshot_prompt_mistral_llama,"""categorie"": Raadsadres}",NoPredictionFormat,raadsadres,22.352761,2024-06-05 10:26:48.086168+02:00,Classificeer het document in één van de catego...,FT_AmsterdamDocClassificationMistral200T2Epoch...,train,test,0
